In [1]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Membuat Daftar Kata**

In [ ]:
word_list = {}
for i in range(0, len(df['clean_text'])):
    sentence = df['clean_text'][i]
    # Tokenizing
    word_token = word_tokenize(sentence)
    # Mengecek setiap token
    for token in word_token:
        # Token yang tidak ditemukan pada word_list bernilai 0
        if token not in word_list:
            word_list[token] = 0
        else:
            word_list[token] += 1

print(word_list)

In [ ]:
len(word_list)

**Mengimpor Data Leksikon**

In [8]:
path = "/content/drive/MyDrive/Project Sistem Pakar/modified_full_lexicon.csv"

# Membaca data leksikon
dict = pd.read_csv(path)

negation_words = ['bukan','tidak','ga','gk']

# Menghapus data yang termasuk kata negasi
dict = dict.drop(dict[(dict['word'] == 'bukan') | (dict['word'] == 'tidak') | (dict['word'] == 'ga') | (dict['word'] == 'gk')].index, axis=0)

# Menyetel ulang indeks data
dict = dict.reset_index(drop=True)

dict.head(10)

,word,weight,number_of_words
0,hai,3,1
1,merekam,2,1
2,ekstensif,3,1
3,paripurna,1,1
4,detail,2,1
5,pernik,3,1
6,belas,2,1
7,welas,4,1
8,kabung,1,1
9,rahayu,4,1


In [9]:
len(dict)

10248

In [10]:
dict['number_of_words'].value_counts()

1    9536
2     686
3      24
4       2
Name: number_of_words, dtype: int64

**Menampilkan Daftar Kata yang Tidak Ditemukan pada Data Leksikon**

In [ ]:
# Menginisialisasi stemmer
stemmer_factory = StemmerFactory()
stemmer = stemmer_factory.create_stemmer()

In [ ]:
dict_list = dict.to_list()
not_included_words = []
for word in word_list.keys():
    # Kata yang tidak ditemukan pada dict_list
    if word not in dict_list:
        # Stemming
        kata_dasar = stemmer.stem(word)
        # Kata dasar yang tidak ditemukan pada dict_list ditambahkan pada not_included_words
        if kata_dasar not in dict_list:
            not_included_words.append(word)

print(not_included_words)

In [ ]:
len(not_included_words)

In [ ]:
# Kata yang tidak ditemukan dan muncul lebih dari 3 kali 
not_included_list = {k: v for (k, v) in word_list.items() if ((k in not_included_words) & (v > 3))}

# Mengurutkan not_included_list berdasarkan jumlah kemunculan terbanyak
sort_items = sorted(not_included_list.items(), key = lambda x: x[1], reverse = True)

# Mengambil 20 nilai not_included_list pertama
sort_items = sort_items[0:20]

# Menampilkan not_included_list yang telah diurutkan
for i in sort_items:
    print(i[0], i[1])

**Menampilkan Daftar Kata pada Grafik**

In [ ]:
# Fungsi untuk memfilter kalimat
def filter_sentence(sentence, key_list):
    # Tokenizing
    word_token = word_tokenize(sentence)
    new_sentence = ''
    # Mengecek setiap word_token
    for token in word_token:
        # Token yang tidak ditemukan pada key_list
        if token not in key_list:
            new_sentence = new_sentence + token + ' '
    return new_sentence

In [ ]:
clean_text = df['clean_text'].copy()

# Memfilter kata yang di-plotting dari negation_words
word_plot = clean_text.apply(lambda x: filter_sentence(x, negation_words))

In [ ]:
wordcloud = WordCloud(width = 800, height = 800, background_color = 'black', max_words = 1000, min_font_size = 20).generate(str(word_plot))

# Plot dengan WordCloud
fig = plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

**Menghitung Nilai Sentimen**

In [ ]:
# Fungsi untuk menghitung nilai sentimen
def calculate_sentiment(index, word_token, word, sentiment):
    # Kata yang terdapat negation_words dibelakangnya memiliki nilai sentimen yang negatif
    if (word_token[index-1] in negation_words):
        sentiment += -dict['weight'][dict_list.index(word)]
    # Menambahkan nilai sentimen
    else:
        sentiment += dict['weight'][dict_list.index(word)]
    
    return sentiment

In [ ]:
sentiment_list = []

# Menghitung nilai sentimen setiap kalimat
for i in range(len(df)):
    sentiment = 0 
    word_token = word_tokenize(df['clean_text'][i])
    
    # Mengecek token pada setiap kalimat
    for token in word_token:
        index = word_token.index(token)
        # Menghitung sentimen pada token yang terdapat pada data leksikon
        if token in dict_list :
            sentiment = calculate_sentiment(index, word_token, token, sentiment)
        else:
            kata_dasar = stemmer.stem(token)
            # Mengecek kata dasar dari token jika tidak terdapat pada data leksikon
            if kata_dasar in dict_list:
                sentiment = calculate_sentiment(index, word_token, kata_dasar, sentiment)
            # Mengecek kombinasi kata dengan kata sebelumnya
            elif(len(word_token) > 1):
                # Kombinasi dengan satu kata sebelumnya
                if(index-1 > -1):
                    combination_one = word_token[index-1] + ' ' + token
                    if (combination_one in dict_list):
                        sentiment= calculate_sentiment(index, word_token, combination_one, sentiment)
                    # Kombinasi dengan dua kata sebelumnya
                    elif(index-2 > -1):
                        combination_two = word_token[index-2] + ' ' + combination_one
                        if combination_two in dict_list:
                            sentiment= calculate_sentiment(index, word_token, combination_two, sentiment)
    
    sentiment_list.append(sentiment)

# Menampilkan seluruh nilai sentimen
print(sentiment_list)

In [ ]:
# Menambahkan sentiment_list pada df
sentiment_array = np.array(sentiment_list).reshape(df.shape[0], 1)
sentiment_data = np.hstack((df, sentiment_array))
new_df = pd.DataFrame(sentiment_data, columns = [...])

In [ ]:
new_df.head(10)